In [1]:

import numpy as np
import os, time
import cv2
import h5py
import imutils
import dlib
from imutils.face_utils import rect_to_bb
from skimage.transform import resize
from scipy.spatial import distance
from keras.models import load_model
from mtcnn.mtcnn import MTCNN
from PIL import Image
from os import listdir
from os.path import isdir

from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray


C:\Users\jackadv\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\jackadv\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jackadv\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Us

In [2]:
valid = "valid/"
compare = "testset/"
face_detect = "mtcnn"

min_faceSzie = (90, 90)

min_score = 0.55
image_size = 160
giveupScore = 0.8
black_padding_width = 0  #add padding width for the face area

make_dataset = False
load_dataset = True
dataset_file = "officedoor7.h5"

In [3]:
from keras.models import model_from_json
 
#facenet model structure: https://github.com/serengil/tensorflow-101/blob/master/model/facenet_model.json
model = model_from_json(open("facenet_model.json", "r").read())
 
#pre-trained weights https://drive.google.com/file/d/1971Xk5RwedbudGgTIrGAL4F7Aifu7id1/view?usp=sharing
model.load_weights('facenet_weights.h5')
 
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
___________________________________________________________________________________________

In [4]:

cascade_path = 'haarcascade_frontalface_default.xml'

#detector = MTCNN()


In [5]:
def prewhiten(x):
    #cv2.imshow("Before", x)
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj

    #cv2.imshow("After", y)
    #cv2.waitKey(0)
    return y


In [6]:

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [7]:
def align_image(filename, required_size=(160, 160)):

# load image from file
    image = Image.open(filename)
# convert to RGB, if needed
    image = image.convert('RGB')
# convert to array
    pixels = asarray(image)
# create the detector, using default weights
    detector = MTCNN()
    

    # detect faces in the image
    results = detector.detect_faces(pixels)
    if results != []:

	# extract the bounding box from the first face
        x1, y1, width, height = results[0]['box']
	# bug fix
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
	# extract the face
        face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
        return face_array



In [8]:

def preProcess(img):
    whitenImg = prewhiten(img)
    whitenImg = whitenImg[np.newaxis, :]
    return whitenImg


In [9]:

def face2name(face, faceEMBS, faceNames):
    #print(len(faceEMBS), len(faceNames))
    imgFace = preProcess(face)
    embs = l2_normalize(np.concatenate(model.predict(imgFace)))

    smallist_id = 0
    smallist_embs = 999
    for id, valid in enumerate(faceEMBS):
        distanceNum = distance.euclidean(embs, valid)
        #if(distanceNum>giveupScore):
        #    smallist_embs = distanceNum
        #    smallist_id = id
        #    print(distanceNum, "--> give up")
            #break
        #else:
        #print("     ", faceNames[id].decode(), distanceNum)
        if(smallist_embs>distanceNum):
            smallist_embs = distanceNum
            smallist_id = id

    print(faceNames[smallist_id].decode(), smallist_embs)
    return smallist_id, faceNames[smallist_id].decode(), smallist_embs


In [10]:

valid_names = []
valid_embs = []


In [11]:
hf = h5py.File(dataset_file, 'r')
valid_names = hf.get('names')
valid_embs = hf.get('embs')

print("HF file loaded, valid names:", valid_names)


HF file loaded, valid names: <HDF5 dataset "names": shape (521,), type "|S5">


In [12]:
import numpy as np
import pandas as pd
#my_df = pd.read_csv("./sample_submission.csv")

In [13]:
x_test_list = []
x_test_id = []
test3="akane"
test4="neru"
test0="rika"
test1="risa"
test2="yui"

if(len(valid_names)>0):
    i = 0
    for folder in os.listdir(compare):
        for img_file in os.listdir(compare+folder):
            i += 1
            filename, file_extension = os.path.splitext(img_file)
            imgCompared = cv2.imread(compare+folder+"/"+img_file)
            xid=os.path.splitext(img_file)[0]
            path=compare+folder+"/"+img_file


            aligned= align_image(path)
            if(aligned is not None): 
                valid_id, valid_name, score = face2name(aligned, valid_embs, valid_names)

                print(" valid_name:    "+valid_name)
                #print(" score:    ",score)                

            else:
                print(" None:    "+path)
                valid_name="akane"
            #print("ID: ", i)
            if valid_name == test0:
                vaild = 0
            elif valid_name == test1:
                vaild = 1               
            elif valid_name == test2:
                vaild = 2
            elif valid_name == test3:
                vaild = 3       
            elif valid_name == test4:
                vaild = 4
            #print("vaild: ", vaild)    
            x_test_list.append( vaild )
            x_test_id.append( xid )

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
 None:    report/test/testset/006F8606F6.png
neru 0.704370379447937
 valid_name:    neru
rika 0.7438644170761108
 valid_name:    rika
neru 0.6555927991867065
 valid_name:    neru
risa 0.5858023166656494
 valid_name:    risa
risa 0.6665592789649963
 valid_name:    risa
neru 0.4337701201438904
 valid_name:    neru
risa 0.6146275997161865
 valid_name:    risa
yui 0.6231399178504944
 valid_name:    yui
risa 0.6096619367599487
 valid_name:    risa
rika 0.5134366750717163
 valid_name:    rika
rika 0.6269536018371582
 valid_name:    rika
risa 0.5208589434623718
 valid_name:    risa
risa 0.568186342716217
 valid_name:    risa
akane 0.6360902190208435
 valid_name:    akane
rika 0.7809593677520752
 valid_name:    rika
risa 0.643888533115387
 valid_name:    risa
yui 0.6604263186454773
 valid_name:    yui
yui 0.6181763410568237
 valid_name:    yui
neru 0.5549533367156982
 valid_name:    neru
neru 0.6162546873092651
 vali

neru 0.4632345139980316
 valid_name:    neru
yui 0.3451091945171356
 valid_name:    yui
risa 0.575619101524353
 valid_name:    risa
akane 0.6615537405014038
 valid_name:    akane
neru 0.5086705088615417
 valid_name:    neru
yui 0.7779746651649475
 valid_name:    yui
risa 0.4482550024986267
 valid_name:    risa
neru 0.4442901611328125
 valid_name:    neru
akane 0.3523256480693817
 valid_name:    akane
yui 0.5094628930091858
 valid_name:    yui
neru 0.6130334734916687
 valid_name:    neru
yui 0.5811027884483337
 valid_name:    yui
neru 0.4890044331550598
 valid_name:    neru
akane 0.7619345188140869
 valid_name:    akane
yui 0.47300493717193604
 valid_name:    yui
risa 0.6632383465766907
 valid_name:    risa
rika 0.6854968667030334
 valid_name:    rika
akane 0.0
 valid_name:    akane
rika 0.6025258898735046
 valid_name:    rika
akane 0.703789472579956
 valid_name:    akane
neru 0.4703216552734375
 valid_name:    neru
risa 0.7989616990089417
 valid_name:    risa
neru 0.7220817804336548
 v

rika 0.5060276389122009
 valid_name:    rika
yui 0.5969541668891907
 valid_name:    yui
rika 0.6339598298072815
 valid_name:    rika
rika 0.6854920983314514
 valid_name:    rika
neru 0.5236000418663025
 valid_name:    neru
 None:    report/test/testset/CF26F3A06A.png
rika 0.582241952419281
 valid_name:    rika
neru 0.5441609621047974
 valid_name:    neru
risa 0.5634447336196899
 valid_name:    risa
yui 0.5627719759941101
 valid_name:    yui
akane 0.7438830137252808
 valid_name:    akane
rika 0.537020742893219
 valid_name:    rika
risa 0.5793660283088684
 valid_name:    risa
akane 0.5545595288276672
 valid_name:    akane
risa 0.48867249488830566
 valid_name:    risa
akane 0.6048900485038757
 valid_name:    akane
neru 0.5847800970077515
 valid_name:    neru
risa 0.650166392326355
 valid_name:    risa
akane 0.3504117727279663
 valid_name:    akane
rika 0.5176621079444885
 valid_name:    rika
akane 0.4192664325237274
 valid_name:    akane
akane 0.655074417591095
 valid_name:    akane
risa 

In [14]:
print(x_test_list)

[3, 4, 0, 4, 1, 1, 4, 1, 2, 1, 0, 0, 1, 1, 3, 0, 1, 2, 2, 4, 4, 3, 2, 4, 4, 4, 4, 0, 0, 0, 4, 2, 4, 2, 4, 1, 2, 3, 4, 2, 2, 4, 1, 0, 3, 0, 2, 4, 4, 4, 1, 2, 3, 3, 2, 4, 4, 2, 4, 2, 4, 1, 2, 0, 4, 0, 4, 1, 2, 2, 2, 4, 4, 4, 0, 0, 1, 1, 0, 2, 4, 4, 1, 2, 0, 2, 4, 2, 0, 4, 4, 0, 0, 4, 4, 4, 0, 4, 3, 4, 2, 3, 4, 0, 1, 3, 3, 1, 0, 4, 2, 1, 0, 0, 1, 4, 3, 3, 0, 4, 3, 4, 0, 3, 4, 3, 3, 2, 0, 4, 4, 4, 0, 2, 2, 4, 4, 4, 4, 2, 0, 4, 0, 0, 4, 1, 4, 4, 2, 4, 4, 4, 0, 2, 1, 1, 4, 2, 1, 0, 2, 2, 3, 4, 1, 4, 1, 3, 3, 2, 0, 1, 4, 0, 0, 4, 0, 1, 4, 2, 1, 3, 4, 2, 1, 4, 3, 2, 4, 2, 4, 3, 2, 1, 0, 3, 0, 3, 4, 1, 4, 0, 0, 1, 0, 3, 3, 2, 2, 3, 1, 2, 0, 3, 4, 3, 3, 1, 1, 3, 2, 4, 3, 1, 4, 4, 4, 1, 4, 4, 3, 1, 4, 1, 2, 0, 2, 4, 3, 4, 2, 3, 3, 3, 4, 0, 3, 4, 1, 1, 1, 2, 3, 3, 1, 4, 0, 1, 4, 2, 4, 4, 4, 3, 3, 4, 4, 3, 4, 3, 3, 1, 2, 1, 2, 0, 0, 3, 2, 3, 0, 4, 2, 4, 4, 0, 4, 4, 0, 4, 0, 1, 4, 1, 4, 1, 4, 1, 4, 4, 4, 0, 3, 4, 1, 4, 4, 3, 1, 3, 2, 3, 2, 2, 3, 2, 0, 4, 0, 4, 1, 4, 0, 3, 4, 4, 0, 2, 1, 3, 4, 2, 4, 

In [15]:
ans_len = len(x_test_list)
print(ans_len)

438


In [16]:
pd.value_counts(x_test_list)

4    136
3     83
0     80
2     71
1     68
dtype: int64

In [17]:
my_df = pd.read_csv("./sample_submission.csv")

In [18]:
submission = {
    'id': x_test_id,
    'class': x_test_list
}
solution = pd.DataFrame(submission)

#make the submission file
solution.to_csv('sample_submission.csv',index=False)
solution.head(10)

,class,id
0,3,006F8606F6
1,4,00C22FC0EA
2,0,00CECD2708
3,4,0125D1C6C3
4,1,0158CDF991
5,1,01693B60FE
6,4,019AE1E661
7,1,01F03AFBFF
8,2,025D0E8105
9,1,030D460C87
